GET A MPS WITH NATIVE BOND DIMENSION 2

In [285]:
import numpy as np
from ncon import ncon
from numpy import linalg as LA 
from scipy.linalg import null_space as NS 

In [286]:
N = 3
mps = np.random.random(2**N) + np.random.random(2**N) * 1j
mps_norm = mps / LA.norm(mps)
psi_norm = mps_norm / LA.norm(mps_norm)
psi_norm_tens = psi_norm.reshape(2,2,2)
print(LA.norm(psi_norm))
print(psi_norm)

1.0
[0.08239122+0.18253254j 0.20263031+0.23576556j 0.11247537+0.1570885j
 0.49650591+0.18852808j 0.4327886 +0.26270614j 0.01166095+0.16245005j
 0.19353504+0.35507024j 0.2172829 +0.22421333j]


In [287]:
mps_sites = [0 for _ in range(N)]


Atemp, Stemp, Vhtemp = LA.svd(mps_norm.reshape(2,2**(N-1)), full_matrices=False)
mps_sites[0] = Atemp.reshape(1,2,2).transpose(0,2,1)  #shape della matrice A_1 ---> A_8 paper è 2,2 - mapping 1 a 1
mps_sites[1] = (np.diag(Stemp)@Vhtemp)/LA.norm(Stemp)

for i in range(1,N-1):
    Atemp, Stemp, Vhtemp = LA.svd(mps_sites[i].reshape(2**2,2**(N-i-1)), full_matrices=False)
    mps_sites[i] = Atemp[:,:2].reshape(2,2,2)
    mps_sites[i+1] = ((np.diag(Stemp[:2])@Vhtemp[:2,:]).reshape(2,-1))/LA.norm(Stemp)

mps_sites[-1] = ((np.diag(Stemp[:2])@Vhtemp[:2,:]).reshape(2,2,1))/LA.norm(Stemp)

mps_2 = ncon([mps_sites[0], mps_sites[1], mps_sites[2]],[[-1,1,-2],[1,2,-3],[2,-4,-5]])
print("Error from MPS to MPS(2):\n",LA.norm(mps_norm.reshape(2,2,2) - mps_2[0,:,:,0,:]) / LA.norm(mps_norm))



Error from MPS to MPS(2):
 1.141124117253262


In [288]:
print(np.round(ncon([mps_sites[0].conj(), mps_sites[0]],[[1,2,-1],[1,2,-2]])))

print(mps_sites[0].shape)
k_a1 = NS(mps_sites[0].reshape(2,2).conj().T)
print(mps_sites[1].shape)
k_a2 = NS(mps_sites[1].reshape(4,2).conj().T)
print(mps_sites[2].shape)
k_a3 = NS(mps_sites[2].reshape(4,1).conj().T)
print(k_a3)

[[1.+0.j 0.+0.j]
 [0.-0.j 1.+0.j]]
(1, 2, 2)
(2, 2, 2)
(2, 2, 1)
[[-0.63661073-8.27020403e-02j -0.3035888 -0.00000000e+00j
   0.29849347+3.87772583e-02j]
 [ 0.74817264-0.00000000e+00j -0.11809906+1.53422385e-02j
   0.11807659+1.73472348e-18j]
 [-0.11809906-1.53422385e-02j  0.94368057-0.00000000e+00j
   0.05537418+7.19365498e-03j]
 [ 0.11807659-1.73472348e-18j  0.05537418-7.19365498e-03j
   0.94463636-0.00000000e+00j]]


In [3]:
import numpy as np
from numpy import linalg as LA
from scipy.linalg import null_space
from ncon import ncon


N = 3
mps = np.random.random(2**N) #+ np.random.random(2**N) * 1j
mps_norm = mps / LA.norm(mps)
psi_norm = mps_norm / LA.norm(mps_norm)
psi = psi_norm.reshape(2,2,2)
print(LA.norm(psi_norm))
print(psi_norm)

mps_sites = [0 for _ in range(N)]

Atemp, Stemp, Vhtemp = LA.svd(psi.reshape(2,2**(N-1)), full_matrices=False)
mps_sites[0] = Atemp.reshape(1,2,2)  #shape della matrice A_1 ---> A_8 paper è 2,2 - mapping 1 a 1
mps_sites[1] = (np.diag(Stemp)@Vhtemp)/LA.norm(Stemp)

for i in range(1,N-1):
    Atemp, Stemp, Vhtemp = LA.svd(mps_sites[i].reshape(2**2,2**(N-i-1)), full_matrices=False)
    mps_sites[i] = Atemp[:,:2].reshape(2,2,2)
    mps_sites[i+1] = ((np.diag(Stemp[:2])@Vhtemp[:2,:]).reshape(2,-1))/LA.norm(Stemp)

mps_sites[-1] = ((np.diag(Stemp[:2])@Vhtemp[:2,:]).reshape(2,2,1))/LA.norm(Stemp)


print("Checking Isometries:")
for site in mps_sites:
    print(np.round((ncon([site,site],[[1,2,-1],[1,2,-2]]))))

print("Converting Rank-3 tensor (j,l,k) in Rank-2 tensor (j*l,k)")
A_sites = [0 for _ in range(N)]
for i in range(len(mps_sites)):
    shape = mps_sites[i].shape
    A_sites[i] = mps_sites[i].reshape(shape[0]*shape[1], shape[2])

NULL = [0 for _ in range(0,N)]
print("Creating null_spaces and G matrix [A, X] where X is such that A^dagX = 0 and X^dagX = I")

G = [0 for _ in range(N)]
for i in range(len(A_sites)):
    k = null_space(A_sites[i].conj().T)
    NULL[i] = k
    G[i] = np.block([[A_sites[i],k]])
    print("------ Site %i-----"%i)
    print("Isomery: \n" ,A_sites[i])
    print("Null_space: \n", k)
    print("Extended: \n", G[i])


G_rank4 = [G[0]]
for Gs in G[1:]:
    G_rank4.append(Gs.reshape(2,2,2,2))

psi_2 = ncon([mps_sites[0],mps_sites[1],mps_sites[2]],[[-1,-2,1],[1,-3,2],[2,-4,-5]])

0.9999999999999998
[0.40344741 0.5049074  0.50379009 0.2339201  0.04703886 0.07145536
 0.5057956  0.10309183]
Checking Isometries:
[[1. 0.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]]
[[1.]]
Converting Rank-3 tensor (j,l,k) in Rank-2 tensor (j*l,k)
Creating null_spaces and G matrix [A, X] where X is such that A^dagX = 0 and X^dagX = I
------ Site 0-----
Isomery: 
 [[-0.88340446 -0.46861131]
 [-0.46861131  0.88340446]]
Null_space: 
 []
Extended: 
 [[-0.88340446 -0.46861131]
 [-0.46861131  0.88340446]]
------ Site 1-----
Isomery: 
 [[-0.6031297   0.68923815]
 [-0.74416651 -0.5243049 ]
 [-0.22752676  0.22973445]
 [ 0.1751638   0.44415902]]
Null_space: 
 [[-0.33005373 -0.22858233]
 [-0.02129643  0.41336067]
 [ 0.94372125 -0.06959782]
 [-0.00109303  0.87865762]]
Extended: 
 [[-0.6031297   0.68923815 -0.33005373 -0.22858233]
 [-0.74416651 -0.5243049  -0.02129643  0.41336067]
 [-0.22752676  0.22973445  0.94372125 -0.06959782]
 [ 0.1751638   0.44415902 -0.00109303  0.87865762]]
------ Site 2-----
Isomery: 
 

In [4]:
print("Checking isometries")
for site in mps_sites:
    print(np.round(ncon([site,site],[[1,2,-1],[1,2,-2]])))

print("\nChecking unitary")
for i in range(len(G)):
    print("Site %i\n"%i, np.round(G[i]@G[i].T))
    print(ncon([G[i], G[i].conjugate()],[[-1,1],[-2,1]]))

Checking isometries
[[1. 0.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]]
[[1.]]

Checking unitary
Site 0
 [[1. 0.]
 [0. 1.]]
[[1.0000000e+00 1.0260533e-16]
 [1.0260533e-16 1.0000000e+00]]
Site 1
 [[ 1. -0.  0.  0.]
 [-0.  1. -0. -0.]
 [ 0. -0.  1.  0.]
 [ 0. -0.  0.  1.]]
[[ 1.00000000e+00 -2.92595144e-16  1.01631004e-16  1.20357642e-16]
 [-2.92595144e-16  1.00000000e+00 -3.19933074e-17 -1.93917316e-16]
 [ 1.01631004e-16 -3.19933074e-17  1.00000000e+00  1.75806647e-17]
 [ 1.20357642e-16 -1.93917316e-16  1.75806647e-17  1.00000000e+00]]
Site 2
 [[ 1.  0.  0. -0.]
 [ 0.  1.  0. -0.]
 [ 0.  0.  1. -0.]
 [-0. -0. -0.  1.]]
[[ 1.00000000e+00  1.63710062e-16  6.42156995e-17 -9.43782360e-17]
 [ 1.63710062e-16  1.00000000e+00  2.58376572e-17 -4.23688781e-17]
 [ 6.42156995e-17  2.58376572e-17  1.00000000e+00 -1.56069072e-17]
 [-9.43782360e-17 -4.23688781e-17 -1.56069072e-17  1.00000000e+00]]


In [5]:

print("\n\nG RANK_4 ---> G2 top 2-qubit gate (mps site 1-2)")
print("\tMPS site 2")
print(mps_sites[2].reshape(4,1))
print(mps_sites[2].shape)
print("\tG_2 matrix")
print(G_rank4[2])
print(G_rank4[2].shape)

print("\tCorrenspondence A_2 == G_2[:,:,0,0]")
print(G_rank4[2][:,:,0,0].reshape(4,1))

print("\n-------\n")
print("G RANK_4 ---> G1 top 2-qubit gate (mps site 0-1)")
print("\tMPS site 1")
print(mps_sites[1].reshape(8,1))
print("\tG_2 matrix")
print(G_rank4[1])
print(G_rank4[1].shape)

print("\tCorrenspondence A_1 == G_1[:,:,0,:]")
print(G_rank4[1][:,:,0,:].reshape(8,1))


print("\n-------\n")
print("G RANK_4 ---> G0 bottom 1-qubit gate")


print("\tCorrenspondence A_0 == G_0")
print(mps_sites[0].reshape(4,1))
print(mps_sites[0].shape)
print(G_rank4[0].reshape(4,1))
print(G_rank4[0].shape)




G RANK_4 ---> G2 top 2-qubit gate (mps site 1-2)
	MPS site 2
[[ 0.80630522]
 [ 0.51516695]
 [ 0.15648645]
 [-0.24492224]]
(2, 2, 1)
	G_2 matrix
[[[[ 0.80630522 -0.51516695]
   [-0.15648645  0.24492224]]

  [[ 0.51516695  0.8530719 ]
   [-0.04463069  0.069853  ]]]


 [[[ 0.15648645 -0.04463069]
   [ 0.98644304  0.02121846]]

  [[-0.24492224  0.069853  ]
   [ 0.02121846  0.96679027]]]]
(2, 2, 2, 2)
	Correnspondence A_2 == G_2[:,:,0,0]
[[ 0.80630522]
 [ 0.51516695]
 [ 0.15648645]
 [-0.24492224]]

-------

G RANK_4 ---> G1 top 2-qubit gate (mps site 0-1)
	MPS site 1
[[-0.6031297 ]
 [ 0.68923815]
 [-0.74416651]
 [-0.5243049 ]
 [-0.22752676]
 [ 0.22973445]
 [ 0.1751638 ]
 [ 0.44415902]]
	G_2 matrix
[[[[-0.6031297   0.68923815]
   [-0.33005373 -0.22858233]]

  [[-0.74416651 -0.5243049 ]
   [-0.02129643  0.41336067]]]


 [[[-0.22752676  0.22973445]
   [ 0.94372125 -0.06959782]]

  [[ 0.1751638   0.44415902]
   [-0.00109303  0.87865762]]]]
(2, 2, 2, 2)
	Correnspondence A_1 == G_1[:,:,0,:]
[[-

CHECK LEFT-CANONICAL CONDITIONS

In [402]:
for i in mps_sites:
    print(ncon([i.conj(),i],[[1,2,-1],[1,2,-2]]))



[[ 1.00000000e+00 -2.41127129e-18]
 [-2.41127129e-18  1.00000000e+00]]
[[ 1.00000000e+00 -6.74982104e-17]
 [-6.74982104e-17  1.00000000e+00]]
[[1.]]


CREAZIONE DELL'MPO

In [16]:
print(ncon([G_rank4[0], G_rank4[0]],[[-1,1],[-2,1]]).reshape(2,2))
print(ncon([G_rank4[1], G_rank4[1]],[[1,2,-1,-2],[1,2,-3,-4]]).reshape(4,4))
print(ncon([G_rank4[2], G_rank4[2]],[[1,2,-1,-2],[1,2,-3,-4]]).reshape(4,4))

print(np.round(ncon([G_rank4[2], G_rank4[2]],[[-1,-2,1,2],[-3,-4,1,2]]).reshape(4,4)))


MPO_UU = ncon([G_rank4[2], G_rank4[1], G_rank4[0], G_rank4[0],G_rank4[1], G_rank4[2]],[[-1,-2,1,2],[-3,1,4,5],[4,6],[7,6],[-6,3,7,5],[-4,-5,3,2]])
MPO = ncon([G_rank4[2],G_rank4[1],G_rank4[0]],[[-1,-2,-4, 1],[1,-3,-5,2],[2,-6]])
print(MPO_UU.reshape(8,8))

MPO_UU2 = ncon([G_rank4[2], G_rank4[1], G_rank4[0], G_rank4[0],G_rank4[1], G_rank4[2]],[[1,2,-2,-3],[3,4,-1,1],[3,5],[6,5],[6,4,-4,7],[7,2,-5,-6]])

MPO_2 = ncon([G_rank4[2], G_rank4[1], G_rank4[0]],[[-2,-3,1,-6],[2,-5,-1,1],[2,-4]])


zero_state = np.zeros(2**N)
zero_state[0] = 1
zero_state = zero_state.reshape(2,2,2)
print(psi_2)
psi_zero_MPO = ncon([zero_state, MPO],[[1,2,3],[1,2,3,-1,-2,-3]])
psi_zero_MPO2 = ncon ([zero_state, MPO_2],[[1,2,3],[1,2,3,-1,-2,-3]])
print(psi_zero_MPO)
print(psi_zero_MPO2)


[[1.0000000e+00 1.0260533e-16]
 [1.0260533e-16 1.0000000e+00]]
[[ 1.00000000e+00  1.55876408e-16 -6.58064804e-17 -6.86669063e-17]
 [ 1.55876408e-16  1.00000000e+00 -3.80730157e-17 -1.82887334e-16]
 [-6.58064804e-17 -3.80730157e-17  1.00000000e+00 -1.22304965e-18]
 [-6.86669063e-17 -1.82887334e-16 -1.22304965e-18  1.00000000e+00]]
[[ 1.00000000e+00  3.71096879e-17 -9.03266464e-18 -1.48005563e-17]
 [ 3.71096879e-17  1.00000000e+00 -1.60380575e-17  1.31422732e-17]
 [-9.03266464e-18 -1.60380575e-17  1.00000000e+00 -1.61390004e-18]
 [-1.48005563e-17  1.31422732e-17 -1.61390004e-18  1.00000000e+00]]
[[ 1.  0.  0. -0.]
 [ 0.  1.  0. -0.]
 [ 0.  0.  1. -0.]
 [-0. -0. -0.  1.]]
[[ 1.00000000e+00  8.32667268e-17  1.61329283e-16 -3.46944695e-17
  -1.66533454e-16  2.77555756e-16  8.32667268e-17 -2.22044605e-16]
 [ 9.71445147e-17  1.00000000e+00  2.77555756e-17  1.90819582e-16
  -2.77555756e-17  6.93889390e-17 -9.02056208e-17 -1.52655666e-16]
 [ 1.28369537e-16  3.46944695e-17  1.00000000e+00  9.714

In [455]:
G_reshaped = [0 for i in range(0,3)]
G_reshaped[0] = mps_sites[0].reshape(2,2)
G_reshaped[1] = np.zeros((4,4))#, dtype=complex)
G_reshaped[1][0,:] = mps_sites[1].reshape(4,2)[:,0]
G_reshaped[1][1,:] = mps_sites[1].reshape(4,2)[:,1]
G_reshaped[1][2,:] = NULL[1][:,0]
G_reshaped[1][3,:] = NULL[1][:,1]
G_reshaped[2] = np.zeros((4,4))#, dtype=complex)
G_reshaped[2][0,:] = mps_sites[2].reshape(4,1)[:,0]
G_reshaped[2][1,:] = NULL[2][:,0]
G_reshaped[2][2,:] = NULL[2][:,1]
G_reshaped[2][3,:] = NULL[2][:,2]

In [456]:
print(ncon([G_reshaped[2], G_reshaped[2]],[[-1,1],[-2,1]]))

[[ 1.00000000e+00 -1.36849590e-16  1.70469437e-17  8.82841899e-18]
 [-1.36849590e-16  1.00000000e+00  2.75486268e-17 -2.15406870e-17]
 [ 1.70469437e-17  2.75486268e-17  1.00000000e+00 -1.09856465e-17]
 [ 8.82841899e-18 -2.15406870e-17 -1.09856465e-17  1.00000000e+00]]


In [457]:
print(ncon([G_reshaped[0],G_reshaped[0].conjugate()],[[-1,1],[-2,1]]))
print(ncon([G_reshaped[1],G_reshaped[1].conjugate()],[[-1,1],[-2,1]]))
print(ncon([G_reshaped[2],G_reshaped[2].conjugate()],[[-1,1],[-2,1]]))

[[ 1.00000000e+00 -2.41127129e-18]
 [-2.41127129e-18  1.00000000e+00]]
[[ 1.00000000e+00 -6.74982104e-17  6.52938606e-17  7.04255474e-17]
 [-6.74982104e-17  1.00000000e+00 -7.19099542e-17 -8.22917979e-18]
 [ 6.52938606e-17 -7.19099542e-17  1.00000000e+00 -4.27381525e-17]
 [ 7.04255474e-17 -8.22917979e-18 -4.27381525e-17  1.00000000e+00]]
[[ 1.00000000e+00 -1.36849590e-16  1.70469437e-17  8.82841899e-18]
 [-1.36849590e-16  1.00000000e+00  2.75486268e-17 -2.15406870e-17]
 [ 1.70469437e-17  2.75486268e-17  1.00000000e+00 -1.09856465e-17]
 [ 8.82841899e-18 -2.15406870e-17 -1.09856465e-17  1.00000000e+00]]


CHECK CORRESPONDENCE MPS TENSOR TO G MATRICES

In [458]:
A  = [0 for _ in range(N)]
for i, tens in enumerate(mps_sites):
    A[i] = tens.reshape(tens.shape[0]*tens.shape[1], tens.shape[2])




[ 0.3827689  -0.62179912 -0.68272828 -0.02712786]
[ 0.3827689  -0.62179912 -0.68272828 -0.02712786]
